Welcome to Pandas for Data Science
Todays agenda:

    Regular Expressions; Conditional Changes
    Aggregate Statistics (Groupby)
    Working with large amounts of data
    

In [ ]:
import pandas as pd

print ('pandas version', pd.__version__)

# Use this data, GDP for countries in Europe
data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/gapminder_gdp_europe.csv')

# Colab
store = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/cal_housing_small.csv')

# Ctrl + Shift + Enter = Specific Line Run
# Ctrl + Enter = All Lines Run

# REGULAR EXPRESSIONS
# Return rows where the country contains the word 'ria' or 'land' # using regular expressions
data[data["country"].str.contains("ria|land")]

# Return rows where the country name starts with a vowel # using regular expressions
data[data["country"].str.contains(r"^[AEIOUaeiou]", na=False)]

# Return rows where the country name contains exactly two vowels # using regular expressions
data[data["country"].str.contains(r"^[^AEIOUaeiou]*[AEIOUaeiou][^AEIOUaeiou]*[AEIOUaeiou][^AEIOUaeiou]*$", na=False)]

# Return rows where the country name contains repeated letters # using regular expressions
data[data["country"].str.contains(r"(.)\1", na=False)]

# Return rows where the country name contains Sp letters at the beginning # using regular expressions
data[data["country"].str.contains(r"(^Sp)", na=False)]

# Return rows where the country name contains ny at the end # using regular expressions
data[data["country"].str.contains(r"(ny$)", na=False)]

# Return rows where the country name contains ria or any at the end # using regular expressions
data[data["country"].str.contains(r"any$|ria$", case=False, na=False)]

# Return rows where the country name contains 8 letters # using regular expressions
data[data["country"].str.contains(r"(^.{8}$)", na=False)]

# Return rows where the country name contains 3 or more vowels # using regular expressions
data[data["country"].str.contains(r"([AEIOUaeiou].*){3,}", na=False)]

# Return rows where the country name contains spaces # using regular expressions
data[data["country"].str.contains(r"[ ]", na=False)]

# Return rows where the country name starts and ends with the same letter # using regular expressions
data[data["country"].str.contains(r"^(.).*\1$", na=False)]

# To turn off regular expressions. Country name contains 3 or more vowels # using regular expressions
data[data["country"].str.contains(r"([AEIOUaeiou].*){3,}", na=False, regex=False)]

# To turn on regular expressions. Country name contains 3 or more vowels # using regular expressions
data[data["country"].str.contains(r"([AEIOUaeiou].*){3,}", na=False, regex=True)]

pandas version 2.1.4


<ipython-input-16-4d49d107ef2e>:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(.)\1", na=False)]
<ipython-input-16-4d49d107ef2e>:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(^Sp)", na=False)]
<ipython-input-16-4d49d107ef2e>:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(ny$)", na=False)]
<ipython-input-16-4d49d107ef2e>:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(^.{8}$)", na=False)]
<ipython-input-16-4d49d107ef2e>:40: UserWarning: This pattern is interpreted as a regular exp

,country,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,gdpPercap_1992,gdpPercap_1997,gdpPercap_2002,gdpPercap_2007
0,Albania,1601.056136,1942.284244,2312.888958,2760.196931,3313.422188,3533.003910,3630.880722,3738.932735,2497.437901,3193.054604,4604.211737,5937.029526
1,Austria,6137.076492,8842.598030,10750.721110,12834.602400,16661.625600,19749.422300,21597.083620,23687.826070,27042.018680,29095.920660,32417.607690,36126.492700
2,Belgium,8343.105127,9714.960623,10991.206760,13149.041190,16672.143560,19117.974480,20979.845890,22525.563080,25575.570690,27561.196630,30485.883750,33692.605080
3,Bosnia and Herzegovina,973.533195,1353.989176,1709.683679,2172.352423,2860.169750,3528.481305,4126.613157,4314.114757,2546.781445,4766.355904,6018.975239,7446.298803
4,Bulgaria,2444.286648,3008.670727,4254.337839,5577.002800,6597.494398,7612.240438,8224.191647,8239.854824,6302.623438,5970.388760,7696.777725,10680.792820
5,Croatia,3119.236520,4338.231617,5477.890018,6960.297861,9164.090127,11305.385170,13221.821840,13822.583940,8447.794873,9875.604515,11628.388950,14619.222720
6,Czech Republic,6876.140250,8256.343918,10136.867130,11399.444890,13108.453600,14800.160620,15377.228550,16310.443400,14297.021220,16048.514240,17596.210220,22833.308510
11,Greece,3530.690067,4916.299889,6017.190733,8513.097016,12724.829570,14195.524280,15268.420890,16120.528390,17541.496340,18747.698140,22514.254800,27538.411880
13,Iceland,7267.688428,9244.001412,10350.159060,13319.895680,15798.063620,19654.962470,23269.607500,26923.206280,25144.392010,28061.099660,31163.201960,36180.789190
14,Ireland,5210.280328,5599.077872,6631.597314,7655.568963,9530.772896,11150.981130,12618.321410,13872.866520,17558.815550,24521.947130,34077.049390,40675.996350
